In [289]:
# Import packages
import matplotlib.pyplot as plt
import nltk
import wordcloud
from wordcloud import WordCloud
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from collections import Counter
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

Import text

In [290]:
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet')

nltk.download('punkt_tab')
    
filename = 'sonnets.txt'
with open(filename, 'r', encoding='utf-8') as file:
    text_data = file.read()

[nltk_data] Downloading package wordnet to /home/eki/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/eki/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Tokenize

In [291]:
tokens = word_tokenize(text_data)

Lemmatizer and stop words

In [292]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

Pre-processed tokens

In [293]:
processed_tokens = [
    lemmatizer.lemmatize(word.lower())
    for word in tokens
    if word.isalpha() and word.lower() not in stop_words and 2 <= len(word) <= 15
]

Bags of words creation

In [294]:
raw_bag = Counter(word_tokenize(text_data))
cleaned_bag = Counter(processed_tokens)

print( processed_tokens )

vectorizer = CountVectorizer(ngram_range=(2, 2))
ngrams = vectorizer.fit_transform([' '.join(processed_tokens)])
ngrams_bag = {key.replace(' ', '-'): value for key, value in zip(vectorizer.get_feature_names_out(), ngrams.toarray().sum(axis=0))}

print( ngrams_bag )

ngrams_df = pd.DataFrame(ngrams_bag.items(), columns=['2-gram', 'Count']).sort_values(by='Count', ascending=False)
print(ngrams_df.head(20))

['sonnet', 'begetter', 'insuing', 'sonnet', 'happiness', 'eternity', 'promised', 'poet', 'wisheth', 'adventurer', 'setting', 'forth', 'fairest', 'creature', 'desire', 'increase', 'thereby', 'beauty', 'rose', 'might', 'never', 'die', 'riper', 'time', 'decease', 'tender', 'heir', 'might', 'bear', 'memory', 'thou', 'contracted', 'thine', 'bright', 'eye', 'thy', 'flame', 'fuel', 'making', 'famine', 'abundance', 'lie', 'thyself', 'thy', 'foe', 'thy', 'sweet', 'self', 'cruel', 'thou', 'art', 'world', 'fresh', 'ornament', 'herald', 'gaudy', 'spring', 'within', 'thine', 'bud', 'buriest', 'thy', 'content', 'tender', 'churl', 'makest', 'waste', 'niggarding', 'pity', 'world', 'else', 'glutton', 'eat', 'world', 'due', 'grave', 'thee', 'ii', 'forty', 'winter', 'shall', 'beseige', 'thy', 'brow', 'dig', 'deep', 'trench', 'thy', 'beauty', 'field', 'thy', 'youth', 'proud', 'livery', 'gazed', 'tatter', 'weed', 'small', 'worth', 'held', 'ask', 'thy', 'beauty', 'lie', 'treasure', 'thy', 'lusty', 'day', 's

In [295]:
# # Visualize word clouds
# plt.figure(figsize=(12, 6))
# plt.subplot(1, 2, 1)
# try:
#     WordCloud(width=400, height=300).generate_from_frequencies(raw_bag).to_image().show()
#     plt.title('Raw Data')
# except NameError:
#     print("WordCloud module is missing, skipping word cloud visualization for raw data.")

# plt.subplot(1, 2, 2)
# try:
#     WordCloud(width=400, height=300).generate_from_frequencies(cleaned_bag).to_image().show()
#     plt.title('Cleaned Data')
# except NameError:
#     print("WordCloud module is missing, skipping word cloud visualization for cleaned data.")

# plt.subplot(1, 3, 3)
# try:
#     WordCloud(width=400, height=300).generate_from_frequencies(ngrams_bag).to_image().show()
#     plt.title('2-gram Data')
# except NameError:
#     print("WordCloud module is missing, skipping word cloud visualization for 2-gram data.")

In [296]:
top_words = pd.DataFrame(cleaned_bag.most_common(20), columns=['Word', 'Count'])
print(top_words)

top_ngrams = pd.DataFrame(sorted(ngrams_bag.items(), key=lambda x: x[1], reverse=True)[:20], columns=['2-gram', 'Count'])
print(top_ngrams)

      Word  Count
0      thy    266
1     thou    234
2     love    196
3     thee    162
4      eye     93
5     doth     88
6     time     78
7   beauty     74
8     mine     63
9    sweet     61
10   heart     61
11   shall     59
12     art     53
13    make     52
14     yet     50
15   thine     44
16     day     43
17     one     43
18    hath     43
19   still     41
        2-gram  Count
0     thou-art     34
1     mine-eye     24
2    thou-dost     13
3    thine-eye     12
4   thy-beauty     12
5     thy-love     12
6    thy-sweet     12
7    dost-thou     10
8    thou-wilt     10
9    love-thee      9
10   thou-hast      9
11   thy-heart      9
12   love-love      8
13  thou-mayst      8
14   love-thou      7
15   thee-thou      7
16  thou-shalt      7
17   thy-worth      7
18   dear-love      6
19  sweet-love      6


In [297]:
# plt.figure()
# plt.bar(top_words['Word'], top_words['Count'])
# plt.title('Top 20 Words')
# plt.xticks(rotation=45)
# plt.show()

# plt.figure()
# plt.bar(top_ngrams['2-gram'], top_ngrams['Count'])
# plt.title('Top 20 2-grams')
# plt.xticks(rotation=45)
# plt.show()

# print("Complete 2-gram Bag of Words:")
# print(ngrams_df)

Processing data for NN input

In [298]:
text_data = text_data.replace("  ", "")

text_data = text_data.split('\n\n')  # Split the text into separate sonnets

text_data = text_data[4::2]  # Select every second entry starting from the 5th


Special characters for NN processing

In [299]:
start_of_text_character = "\x0002"
whitespace_character = "\x00B7"
end_of_text_character = "\x2403"
newline_character = "\x00B6"

Start char & spaces/ new lines

In [300]:
text_data = [start_of_text_character + text for text in text_data]


text_data = [text.replace(" ", whitespace_character).replace("\n", newline_character) for text in text_data]


Unique chars and sentence prep

In [301]:
unique_characters = sorted(set(''.join(text_data)))
num_unique_characters = len(unique_characters)
num_documents = len(text_data)
XTrain = []
YTrain = []

In [302]:
for i, doc in enumerate(text_data):
    sequence_length = len(doc)
    idx = [unique_characters.index(char) for char in doc]
    X = np.zeros((num_unique_characters, sequence_length))
    for j, index in enumerate(idx):
        X[index, j] = 1
    
    characters_shifted = list(doc[1:]) + [end_of_text_character]
    Y = pd.Categorical(characters_shifted, categories=unique_characters)
    XTrain.append(X)
    YTrain.append(Y)

# print(text_data[0])  # Display the first processed document
# print(YTrain[0])  # Display the first YTrain sequence